In [62]:
import pandas as pd
import datetime as dt
import hvplot.pandas
import warnings

# Ignore specific FutureWarnings related to holoviews data handling
warnings.filterwarnings('ignore', category=FutureWarning, module='holoviews.core.data')
warnings.filterwarnings('ignore', category=FutureWarning, module='holoviews.core.util')

In [2]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [3]:
coin_list = cg.get_coins_list()

coins = pd.DataFrame(coin_list)

coins

,id,symbol,name
0,01coin,zoc,01coin
1,0chain,zcn,Zus
2,0-knowledge-network,0kn,0 Knowledge Network
3,0-mee,ome,O-MEE
4,0vix-protocol,vix,0VIX Protocol
...,...,...,...
14370,zyncoin-2,zyn,ZynCoin
14371,zynecoin,zyn,Zynecoin
14372,zynergy,zyn,Zynergy
14373,zyrri,zyr,Zyrri


In [11]:
token_id = 'token-engineering-commons'
quote = 'ETH'
days = 365

def get_price_history(token_id: str, quote: str, days: int) -> pd.DataFrame:
    # Pull prices from coin gecko
    prices = cg.get_coin_market_chart_by_id(token_id, quote, days)
    
    # Little bit of clean up
    prices['timestamp'] = [[p[0],p[0]] for p in prices['prices']]
    prices = {a:[i[1] for i in b] for a,b in prices.items()}
    
    # Make a dataframe time series
    prices_df = pd.DataFrame(prices)
    
    prices_df['token_id'] = token_id
    
    return prices_df

# Make a dataframe time series
prices_df = get_price_history(token_id, quote, days)

prices_df

,prices,market_caps,total_volumes,timestamp,token_id
0,0.000289,0.0,0.054882,1684886400000,token-engineering-commons
1,0.000292,0.0,0.197983,1684972800000,token-engineering-commons
2,0.000286,0.0,0.115683,1685059200000,token-engineering-commons
3,0.000285,0.0,0.018294,1685145600000,token-engineering-commons
4,0.000287,0.0,0.054264,1685232000000,token-engineering-commons
...,...,...,...,...,...
241,0.000165,0.0,0.373036,1716076800000,token-engineering-commons
242,0.000145,0.0,0.182577,1716163200000,token-engineering-commons
243,0.000142,0.0,0.010376,1716249600000,token-engineering-commons
244,0.000149,0.0,1.208917,1716336000000,token-engineering-commons


In [14]:
tec_data = pd.read_csv('data/TEC-prices.csv')

tec_data['timestamp'] = pd.to_datetime(tec_data['Date (00:00 UTC,Coingecko)']).dt.normalize()

tec_data['prices'] = tec_data['TEC price (ETH)']
tec_data['token_id'] = 'token-engineering-commons'
tec_data['market_caps'] = 0
tec_data['total_volumes'] = 0

tec_data = tec_data[prices_df[prices_df['token_id']=='token-engineering-commons'].columns]

tec_data

,prices,market_caps,total_volumes,timestamp,token_id
0,0.000514,0,0,2021-07-14,token-engineering-commons
1,0.000501,0,0,2021-07-15,token-engineering-commons
2,0.000523,0,0,2021-07-16,token-engineering-commons
3,0.000534,0,0,2021-07-17,token-engineering-commons
4,0.000526,0,0,2021-07-18,token-engineering-commons
...,...,...,...,...,...
724,0.000269,0,0,2023-07-08,token-engineering-commons
725,0.000266,0,0,2023-07-09,token-engineering-commons
726,0.000271,0,0,2023-07-10,token-engineering-commons
727,0.000265,0,0,2023-07-11,token-engineering-commons


In [15]:
token_ids = [
    'bright-token',
    'gitcoin',
    'giveth',
    'rad',
    'honey-finance',
    'token-engineering-commons',
    'metagame',
    
]

In [16]:
data = []
for token_id in token_ids:
    print(f"Fetching data for token_id: {token_id}")
    df = get_price_history(token_id, quote='ETH', days=days)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms').dt.normalize()

    if token_id == 'token-engineering-commons':
        df = df.set_index('timestamp').combine_first(tec_data.set_index('timestamp')).reset_index()
    data.append(df)

print(f"Fetching data for ETH/USD")
df = get_price_history('ethereum', quote='usd', days=days)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')#.dt.normalize()
data.append(df)

Fetching data for token_id: bright-token
Fetching data for token_id: gitcoin
Fetching data for token_id: giveth
Fetching data for token_id: rad
Fetching data for token_id: honey-finance
Fetching data for token_id: token-engineering-commons
Fetching data for token_id: metagame
Fetching data for ETH/USD


In [17]:
# Concatenate all of the data into one dataframe
prices_df = pd.concat(data)
prices_df

,prices,market_caps,total_volumes,timestamp,token_id
0,0.000023,0.000000e+00,1.747378e-01,2023-05-24,bright-token
1,0.000021,0.000000e+00,4.439057e-02,2023-05-25,bright-token
2,0.000021,0.000000e+00,3.869554e-02,2023-05-26,bright-token
3,0.000020,0.000000e+00,4.981101e-02,2023-05-27,bright-token
4,0.000020,0.000000e+00,2.633131e-02,2023-05-28,bright-token
...,...,...,...,...,...
361,3120.549378,3.752063e+11,8.357435e+09,2024-05-19,ethereum
362,3071.324418,3.685981e+11,6.983634e+09,2024-05-20,ethereum
363,3656.390090,4.398307e+11,2.715950e+10,2024-05-21,ethereum
364,3792.486220,4.557655e+11,3.737006e+10,2024-05-22,ethereum


In [35]:
prices_df[prices_df['token_id']=='token-engineering-commons']

,prices,market_caps,total_volumes,timestamp,token_id
0,0.000514,0.0,0.000000,2021-07-14,token-engineering-commons
1,0.000501,0.0,0.000000,2021-07-15,token-engineering-commons
2,0.000523,0.0,0.000000,2021-07-16,token-engineering-commons
3,0.000534,0.0,0.000000,2021-07-17,token-engineering-commons
4,0.000526,0.0,0.000000,2021-07-18,token-engineering-commons
...,...,...,...,...,...
920,0.000165,0.0,0.373036,2024-05-19,token-engineering-commons
921,0.000145,0.0,0.182577,2024-05-20,token-engineering-commons
922,0.000142,0.0,0.010376,2024-05-21,token-engineering-commons
923,0.000149,0.0,1.208917,2024-05-22,token-engineering-commons


In [22]:
prices_df[prices_df['token_id']=='token-engineering-commons'].hvplot.step(x='timestamp', y='prices', by='token_id')

:NdOverlay   [token_id]
   :Curve   [timestamp]   (prices)

In [50]:
historic_data = pd.read_csv('data/public_goods_prices.csv').drop('normalized_prices',axis=1)
historic_data['timestamp'] = pd.to_datetime(historic_data['timestamp']).dt.normalize()

In [54]:
combined_df = prices_df.set_index(['timestamp','token_id']).combine_first(historic_data.set_index(['timestamp','token_id'])).reset_index()

In [124]:
combined_df.to_csv('data/combined_updated.csv', index=False)